In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os

def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def get_csi_amplitudes(csi_value):
    csi_amplitudes = []
    for v in csi_value:
        amplitude = np.absolute(v)
        csi_amplitudes.append(amplitude)
    return csi_amplitudes

def get_data(mat_filename, transmitter, receiver, authenticated_pair = [' ',' ']):
    annots = loadmat(mat_filename)
    data = []
    for i in range(len(annots['Raw_Cell_Matrix'])):
        authenticated = 0
        if transmitter == authenticated_pair[0] and receiver == authenticated_pair[1]:
            authenticated = 1
        data.append([
            int(annots['Raw_Cell_Matrix'][i][0]['timestamp_low'][0][0][0][0]),
            transmitter,
            receiver,
            annots['Raw_Cell_Matrix'][i][0]['agc'][0][0][0][0]
            ] + get_csi_amplitudes(annots['Raw_Cell_Matrix'][i][0]['CSI'][0][0][0][0])
            + [authenticated]
            )

    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns + ['Authenticated']
    final_data = pd.DataFrame(data, columns=columns)
    return final_data

# print(prepare_data('data.mat','S13','S21'))

from pathlib import Path

def prepare_data(directory, authenticated_pair):
    csi_columns = ['A'+ str(i) for i in range(1,31)]
    columns = ['timestamp', 'transmitter', 'receiver', 'RSS'] + csi_columns
    all_data = pd.DataFrame([], columns=columns)
    for p in getListOfFiles(Path(directory)):
        pair = p.split('/')[1]
        print(p, pair.split('_')[0], pair.split('_')[1])
        data = get_data(p, pair.split('_')[0], pair.split('_')[1], authenticated_pair = authenticated_pair)
        all_data = pd.concat([all_data, data])
    return all_data



In [2]:
authenticated_pair = ['S13', 'S21']

all_data = prepare_data('./data', authenticated_pair)
print(all_data.info())

data/S19_S11/I5/S19_S11_I5_T1.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T8.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T7.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T6.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T10.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T2.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T4.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T5.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T3.mat S19 S11
data/S19_S11/I5/S19_S11_I5_T9.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T7.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T8.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T10.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T1.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T5.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T3.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T9.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T6.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T4.mat S19 S11
data/S19_S11/I3/S19_S11_I3_T2.mat S19 S11
data/S19_S11/I2/S19_S11_I2_T8.mat S19 S11
data/S19_S11/I2/S19_S11_I2_T7.mat S19 S11
data/S19_S11/I2/S19_S11_I2_T1.mat S19 S11
data/S19_S11/I2/S19_S11_I2_T10.m

In [3]:
all_data.head()

,timestamp,transmitter,receiver,RSS,A1,A2,A3,A4,A5,A6,...,A22,A23,A24,A25,A26,A27,A28,A29,A30,Authenticated
0,1745689377,S19,S11,41,32.557641,35.805028,40.706265,48.259714,53.254108,53.712196,...,44.045431,45.177428,55.803226,54.083269,52.924474,46.227697,36.124784,38.639358,35.608988,0.0
1,1745692081,S19,S11,41,26.172505,31.016125,31.016125,47.381431,47.853944,41.725292,...,39.357337,41.194660,48.548944,50.219518,47.169906,39.293765,38.327536,36.124784,29.832868,0.0
2,1745695172,S19,S11,42,39.051248,40.162171,42.426407,46.861498,59.774577,53.758720,...,43.737855,48.877398,53.366656,53.758720,55.443665,44.384682,40.261644,42.011903,35.468296,0.0
3,1745698273,S19,S11,42,31.064449,32.249031,38.600518,43.600459,57.558666,48.754487,...,38.288379,47.801674,48.600412,55.081757,51.039201,41.785165,33.541020,35.510562,33.541020,0.0
4,1745701173,S19,S11,41,32.557641,43.416587,41.340053,50.537115,60.000000,56.938563,...,45.398238,53.254108,58.051701,55.901699,57.008771,49.477268,41.036569,40.816663,38.013156,0.0


In [4]:
all_data.loc[all_data['Authenticated'] == 1]

,timestamp,transmitter,receiver,RSS,A1,A2,A3,A4,A5,A6,...,A22,A23,A24,A25,A26,A27,A28,A29,A30,Authenticated
0,1700042543,S13,S21,39,16.763055,26.248809,28.600699,29.154759,38.209946,30.675723,...,22.472205,22.360680,27.802878,27.892651,22.203603,21.189620,17.492856,15.652476,16.000000,1.0
1,1700045404,S13,S21,39,14.035669,25.632011,30.364453,27.202941,34.409301,32.695565,...,24.596748,21.023796,17.492856,24.166092,27.166155,21.213203,15.231546,15.231546,16.031220,1.0
2,1700048399,S13,S21,38,19.104973,21.931712,26.172505,26.627054,31.144823,35.227830,...,18.357560,20.024984,25.495098,28.231188,23.021729,19.104973,20.615528,18.439089,17.029386,1.0
3,1700051409,S13,S21,39,26.476405,29.614186,25.179357,33.615473,36.235342,32.893768,...,23.021729,22.203603,30.675723,29.410882,21.931712,19.723083,21.095023,21.587033,18.439089,1.0
4,1700054450,S13,S21,39,22.825424,18.384776,19.209373,30.805844,39.560081,33.060551,...,18.027756,21.840330,25.612497,28.017851,18.027756,17.464249,22.671568,19.235384,13.928388,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,18446744072968511488,S13,S21,42,36.069378,33.541020,35.383612,48.373546,42.953463,47.675990,...,28.071338,27.018512,28.071338,29.410882,31.112698,30.463092,22.022716,25.317978,22.671568,1.0
1794,18446744072968513536,S13,S21,42,34.525353,38.275318,34.176015,42.059482,51.400389,46.324939,...,28.231188,24.698178,28.160256,31.112698,33.105891,28.284271,25.806976,25.495098,23.537205,1.0
1795,18446744072968517632,S13,S21,41,33.015148,32.202484,42.011903,49.979996,52.697249,52.430907,...,29.427878,26.419690,33.120990,38.470768,31.622777,26.627054,26.248809,29.017236,28.301943,1.0
1796,18446744072968519680,S13,S21,42,30.413813,38.948684,32.249031,41.436699,51.039201,49.040799,...,27.892651,27.730849,27.073973,35.777088,31.400637,25.709920,23.194827,22.360680,21.213203,1.0


In [5]:
df = all_data.drop(['timestamp','transmitter', 'receiver','Authenticated'],axis = 1)
dfy = all_data[['Authenticated']]

In [6]:
df.head()
print('----')
dfy.head()

----


,Authenticated
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [7]:
from sklearn.neighbors import KNeighborsClassifier

X_train = df
y_train = dfy

knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)
y_train.shape

/home/hadi/Desktop/testing ai/dataset mat/venv/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(331271, 1)

In [8]:
# Save model
import pickle
filename = 'knn_model_class.sav'
pickle.dump(knn, open(filename, 'wb'))

In [11]:
# print(df.iloc[1])
# test = df.loc[df['Authenticated'] == True]
df_list = df.values.tolist()


In [12]:
test = df_list[40]
print(test)

[41, 31.827660925679098, 37.21558813185679, 39.698866482558415, 42.15447781671598, 62.96824596572466, 53.46026561849464, 50.20956084253277, 43.56604182158393, 44.41846462902562, 41.14608122288197, 47.53945729601885, 51.10772935672255, 50.635955604688654, 57.3149195236284, 58.18934610390462, 63.00793600809346, 52.0, 51.088159097779204, 50.44799302251776, 45.617978911828175, 41.23105625617661, 39.35733730830886, 48.373546489791295, 50.35871324805669, 59.481089431852205, 51.62363799656123, 42.95346318982906, 37.05401462729781, 38.47076812334269, 34.20526275297414]


In [13]:
print(regressor.predict(np.array(test).reshape(1, -1)))


[0.03]


/home/hadi/Desktop/testing ai/dataset mat/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [14]:
true = all_data.loc[all_data['Authenticated'] == True]
df_list_true = true.values.tolist()



In [15]:
true_test = df_list_true[0][3:34]
print(true_test)


[39, 16.76305461424021, 26.248809496813376, 28.600699292150182, 29.154759474226502, 38.2099463490856, 30.675723300355934, 34.828149534535996, 28.0178514522438, 29.546573405388315, 23.08679276123039, 25.179356624028344, 27.586228448267445, 32.2490309931942, 31.78049716414141, 30.14962686336267, 32.89376840679705, 31.064449134018133, 26.476404589747453, 21.400934559032695, 24.351591323771842, 24.186773244895647, 22.47220505424423, 22.360679774997898, 27.80287754891569, 27.892651361962706, 22.20360331117452, 21.18962010041709, 17.4928556845359, 15.652475842498529, 16.0]


In [16]:
print(regressor.predict(np.array(true_test).reshape(1, -1)))

[0.97]


/home/hadi/Desktop/testing ai/dataset mat/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [17]:
regressor.score(X_train, y_train)

0.9864569792292481

In [18]:
authenticated_pair = ['S13', 'S21']
test_data = prepare_data('./test_data',authenticated_pair = authenticated_pair)

test_data/S18_S57/I5/S18_S57_I5_T5.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T3.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T8.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T2.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T1.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T6.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T7.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T4.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T10.mat S18 S57
test_data/S18_S57/I5/S18_S57_I5_T9.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T3.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T5.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T8.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T4.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T7.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T10.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T1.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T6.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T9.mat S18 S57
test_data/S18_S57/I3/S18_S57_I3_T2.mat S18 S57
test_data/S18_S57/I2/S18_S57_I2_T10.mat S18 S57
test_data/

In [19]:
df_test = test_data.drop(['timestamp','transmitter', 'receiver','Authenticated'],axis = 1)
dfy_test = test_data[['Authenticated']]

In [20]:
X_test = df_test
Y_test = dfy_test
Y_test.head()

,Authenticated
0,False
1,False
2,False
3,False
4,False


In [21]:
Y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218933 entries, 0 to 1648
Data columns (total 1 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Authenticated  218933 non-null  object
dtypes: object(1)
memory usage: 3.3+ MB


In [22]:
regressor.score(X_test, Y_test)

0.0

In [23]:
test = df_list[40]
print(test)
print(np.array(test).reshape(1, -1))
print(regressor.predict(np.array(true_test).reshape(1, -1)))


[41, 31.827660925679098, 37.21558813185679, 39.698866482558415, 42.15447781671598, 62.96824596572466, 53.46026561849464, 50.20956084253277, 43.56604182158393, 44.41846462902562, 41.14608122288197, 47.53945729601885, 51.10772935672255, 50.635955604688654, 57.3149195236284, 58.18934610390462, 63.00793600809346, 52.0, 51.088159097779204, 50.44799302251776, 45.617978911828175, 41.23105625617661, 39.35733730830886, 48.373546489791295, 50.35871324805669, 59.481089431852205, 51.62363799656123, 42.95346318982906, 37.05401462729781, 38.47076812334269, 34.20526275297414]
[[41.         31.82766093 37.21558813 39.69886648 42.15447782 62.96824597
  53.46026562 50.20956084 43.56604182 44.41846463 41.14608122 47.5394573
  51.10772936 50.6359556  57.31491952 58.1893461  63.00793601 52.
  51.0881591  50.44799302 45.61797891 41.23105626 39.35733731 48.37354649
  50.35871325 59.48108943 51.623638   42.95346319 37.05401463 38.47076812
  34.20526275]]
[0.97]


/home/hadi/Desktop/testing ai/dataset mat/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [32]:
y_pred = regressor.predict(X_test)
df1=pd.DataFrame(data=[[Y_test,y_pred]],columns=['Actual','Predicted'])
print(y_pred)
print(Y_test)

[0.17 0.14 0.19 ... 0.28 0.16 0.46]
     Authenticated
0            False
1            False
2            False
3            False
4            False
...            ...
1644         False
1645         False
1646         False
1647         False
1648         False

[218933 rows x 1 columns]


In [ ]:
import random
import time
while True:
    rss = [random.randint(40, 90)]
    csi_amplitude = [random.randint(40, 90) for i in range(1,31)]
    test = rss + csi_amplitude
    time.sleep(3)
    print(regressor.predict(np.array(test).reshape(1, -1)) > 0.5)